In [27]:
import os
import gym
import ray
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.gym.environment import AnimalAIGym
from ray.rllib.agents import ppo

In [28]:
from ray.tune import register_env, tune

In [40]:
import sys  
sys.path.insert(0, '../')
import cognitive
from cognitive.primitive_arena import RayAIIGym


In [30]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
from ray.tune.logger import pretty_print

from cache_model import *
from config import get_cfg
from custom_model import *

In [31]:
class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('/home/azibit/Downloads/AnimalAI-Olympics/examples/configurations/curriculum/0.yml')
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [55]:
conf = {
            "num_workers": 0,
            "env_config": {
                "unity_worker_id": 55
            },
            "model": {
                    "custom_model": 'my_cnn_rnn_model',
                    "custom_model_config": {},
                },
        "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
        "num_workers": 1,  # parallelism
        "framework": "torch",
        "train_batch_size": 500
       }

In [56]:
conf

{'num_workers': 1,
 'env_config': {'unity_worker_id': 55},
 'model': {'custom_model': 'my_cnn_rnn_model', 'custom_model_config': {}},
 'num_gpus': 0,
 'framework': 'torch',
 'train_batch_size': 500}

In [57]:
ray.shutdown()
ray.init()

# Register custom models so that we can give the ID to the policy trainer
# ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

register_env("unity_env", lambda config: UnityEnvWrapper(config))

2021-03-28 17:59:32,931	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


In [47]:
result = tune.run("PPO", stop = {"timesteps_total": 1000, "training_iteration": 3}, config = conf, checkpoint_at_end = True)

Trial name,status,loc
PPO_unity_env_57e66_00000,RUNNING,


(pid=428119) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=428119) Instructions for updating:
(pid=428119) non-resource variables are not supported in the long term
(pid=428119) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=428119) Instructions for updating:
(pid=428119) non-resource variables are not supported in the long term
(pid=428119) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future vers

(pid=428117) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=428117) Mono path[0] = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/Managed'
(pid=428117) Mono config path = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=428117) Preloaded 'lib_burst_generated.so'
(pid=428117) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=428117) Preloaded 'ScreenSelector.so'
(pid=428117) Display 0 '0': 1920x1080 (primary device).
(pid=428117) Display 1 'C32F391 32"': 1920x1080 (secondary device).
(pid=428117) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log
(pid=428117) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=428117) Mono path[0] = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/Managed'
(pid=428117) Mono config path = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/MonoBlee

(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=428117) INFO:mlagents_envs:Connected to Unity environment with package 

(pid=428119) 2021-03-28 16:27:13,235	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!
(pid=428119) 2021-03-28 16:27:13,235	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!
(pid=428119) 2021-03-28 16:27:13,235	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!
(pid=428119) 2021-03-28 16:27:13,235	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!
(pid=428119) 2021-03-28 16:27:13,235	WARNING

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_unity_env_57e66_00000,RUNNING,10.26.177.163:428119,1,20.0583,600,0.385762,0.968333,-1,82.2857


Result for PPO_unity_env_57e66_00000:
  custom_metrics: {}
  date: 2021-03-28_16-27-53
  done: true
  episode_len_mean: 84.57142857142857
  episode_reward_max: 0.9683333626016974
  episode_reward_mean: 0.376523839975042
  episode_reward_min: -0.9999999310821295
  episodes_this_iter: 7
  episodes_total: 14
  experiment_id: d31d2bf298684bdfa755cb5d33552a8e
  hostname: azibit-Lenovo-Y520-15IKBM
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.30000000000000004
        cur_lr: 5.0e-05
        entropy: 2.1112406849861145
        entropy_coeff: 0.0
        kl: 0.01767533551901579
        policy_loss: -0.07915403507649899
        total_loss: -0.02922076522372663
        vf_explained_var: 0.21328945457935333
        vf_loss: 0.044630685122683644
    num_steps_sampled: 1200
    num_steps_trained: 1200
  iterations_since_restore: 2
  node_ip: 10.26.177.163
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 23.3344827

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_unity_env_57e66_00000,RUNNING,10.26.177.163:428119,2,40.3662,1200,0.376524,0.968333,-1,84.5714


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_unity_env_57e66_00000,TERMINATED,,2,40.3662,1200,0.376524,0.968333,-1,84.5714


2021-03-28 16:27:53,994	INFO tune.py:450 -- Total run time: 48.71 seconds (48.40 seconds for the tuning loop).
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.
(pid=428117) INFO:mlagents_envs:Environment shut down with return code 0.


In [58]:
trainer = PPOTrainer(config=conf, env= "unity_env")

(pid=434444) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=434444) Instructions for updating:
(pid=434444) non-resource variables are not supported in the long term
(pid=434444) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=434444) Instructions for updating:
(pid=434444) non-resource variables are not supported in the long term
(pid=434444) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future vers

(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=434444) Mono path[0] = '/home/azibit/Download

(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=434444) INFO:mlagents_envs:Connected to Unity environment with package 

In [60]:
result = trainer.train()

In [62]:
result

{'episode_reward_max': 0.9710000064224005,
 'episode_reward_min': -0.9999999310821295,
 'episode_reward_mean': 0.12954170745797455,
 'episode_len_mean': 123.875,
 'episodes_this_iter': 5,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [-0.9999999310821295,
   0.9563333634287119,
   0.9710000064224005,
   0.5043333945795894,
   0.9136666627600789,
   0.6910000257194042,
   -0.9999999310821295,
   -0.9999999310821295],
  'episode_lengths': [250, 11, 7, 124, 22, 77, 250, 250]},
 'sampler_perf': {'mean_env_wait_ms': 5.505973566284681,
  'mean_raw_obs_processing_ms': 0.1483802052782564,
  'mean_inference_ms': 1.950556930253454,
  'mean_action_processing_ms': 0.05622616029419605},
 'off_policy_estimator': {},
 'num_healthy_workers': 1,
 'timesteps_total': 1200,
 'timers': {'sample_time_ms': 4606.281,
  'sample_throughput': 130.257,
  'learn_time_ms': 14137.697,
  'learn_throughput': 42.44,
  'update_time

In [49]:
tune.save('Result')

AttributeError: module 'ray.tune.tune' has no attribute 'save'

In [41]:
class RayAIIGym(AnimalAIGym):
    def __init__(self, env_config, arena_config):
        super(RayAIIGym, self).__init__(worker_id=env_config.worker_index,
                                        environment_filename='../examples/env/AnimalAI',
                                        arenas_configurations=arena_config)

In [43]:
ray.shutdown()
ray.init(include_dashboard=False)
register_env("my_env", lambda c: RayAIIGym(c, ArenaConfig('../examples/configurations/curriculum/0.yml')))
ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)
trainer = PPOTrainer(env="my_env", config={
        "model": {
            "custom_model": 'my_cnn_rnn_model',
            "custom_model_config": {},
        },
        "framework": 'torch',
    })
trainer.train()

(pid=426293) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=426293) Instructions for updating:
(pid=426293) non-resource variables are not supported in the long term
(pid=426293) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=426293) Instructions for updating:
(pid=426293) non-resource variables are not supported in the long term
(pid=426293) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future vers

(pid=426293) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=426293) no boot config - using default values
(pid=426293)  
(pid=426293) (Filename:  Line: 495)
(pid=426293) 
(pid=426293) There is no data folder
(pid=426293) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=426293) no boot config - using default values
(pid=426293)  
(pid=426293) (Filename:  Line: 495)
(pid=426293) 
(pid=426293) There is no data folder
(pid=426293) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=426293) no boot config - using default values
(pid=426293)  
(pid=426293) (Filename:  Line: 495)
(pid=426293) 
(pid=426293) There is no data folder
(pid=426293) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=426293) no boot config - using default values
(pid=426293)  
(pid=426293) (Filename:  Line: 495)
(pid=426293) 
(pid=4262

(pid=426293) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426293) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426293) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426293) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426293) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426293) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426293) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426290) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426290) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426290) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426290) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426290) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426290) INFO:mlagents_envs:Environment shut down with return code 1.
(pid=426290) INFO:mlagents_envs:Enviro

RayTaskError(UnityTimeOutException): [36mray::RolloutWorker.foreach_policy()[39m (pid=426290, ip=10.26.177.163)
  File "python/ray/_raylet.pyx", line 439, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 473, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 476, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 480, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 432, in ray._raylet.execute_task.function_executor
  File "/home/azibit/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 378, in __init__
    self.env = _validate_env(env_creator(env_context))
  File "<ipython-input-43-ba7d8f9c1761>", line 3, in <lambda>
  File "<ipython-input-41-0a49311ead4c>", line 5, in __init__
  File "/home/azibit/anaconda3/lib/python3.7/site-packages/animalai/envs/gym/environment.py", line 56, in __init__
    grayscale=grayscale,
  File "/home/azibit/anaconda3/lib/python3.7/site-packages/animalai/envs